In [1]:
import os
import dotenv

dotenv.load_dotenv(override=True)

# override input to mitigate jupyter's buffering

import builtins

def input(*args, **kwargs):
    print(*args, end='', flush=True)
    return builtins.input(*args, **kwargs)

input.__doc__ = builtins.input.__doc__

In [2]:
from langchain_groq import ChatGroq

model_name = 'qwen-qwq-32b'
llm = ChatGroq(
    model=model_name,
    temperature=0.4,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# from langchain_gigachat.chat_models import GigaChat

# llm = GigaChat(
#     credentials=os.environ["GC_AUTH_KEY"],
#     scope="GIGACHAT_API_PERS",
#     model="GigaChat",
#     verify_ssl_certs=False,
# )

In [3]:
from typing_extensions import TypedDict
from langchain.agents import tool, AgentExecutor
from langgraph.prebuilt import create_react_agent

profile_saved = False

class ProfileData(TypedDict):
    name: str
    age: int
    genre: str
    color: str
    city: str

@tool
def validate_city(city: str) -> str:
    """Проверяет, существует ли город с таким названием.
    Если нет, то возвращает ошибку.
    Если есть, то возвращает сообщение об успешной проверке.
    """
    print("Вызываем тул для проверки существования города")
    cities = ["Москва", "Санкт-Петербург", "Казань", "Екатеринбург", "Новосибирск", "Владивосток"]
    if city not in cities:
        return f"❌ Ошибка: города с названием '{city}' не существует."
    return f"✅ Город '{city}' существует."
    

@tool
def save_profile(data: ProfileData) -> str:
    """Сохраняет профиль пользователя.
    Профиль пользователя состоит из следующих полей:
    - name: Имя
    - age: Возраст - целое число больше нуля. Если пользователь ввёл строку или не число, то это неверный ответ. Например, 2.5 или "-2" - это не возраст.
    - genre: Любимый жанр книг
    - city: Любимый город
    - color:Любимый цвет
    
    Возвращает сообщение об успешном сохранении профиля.
    """
    global profile_saved
    print("\n✅ Все данные собраны:")
    for k, v in data.items():
        print(f"- {k}: {v}")
    profile_saved = True
    return "Профиль успешно сохранён."

tools = [save_profile, validate_city]

agent = create_react_agent(
    model=llm,
    tools=tools,
    prompt='''Ты - дружелюбный помощник, который поэтапно собирает информацию от пользователя.
Твоя задача: получить следующие данные:
- Имя - любое имя. Если пользователь ввёл число или другой неожиданный ответ, спроси еще раз.
- Возраст - должно быть целое число больше нуля. Должен быть правдоподобным!
- Любимый город пользователя. Проверь, что город существует! Если нет, спроси еще раз.
- Любимый жанр книг
- Любимый цвет

Ты должен задавать вопросы один за другим. Если ответ некорректный (например, возраст не число),
объясни это и попроси повторить. Когда все данные собраны, сохрани полученные данные.
Помни, ведешь диалог ты, вовлекай пользователя в диалог, не позволяй беседе провисать.'''
)

In [4]:
from langchain_core.messages import HumanMessage, AIMessage

profile_saved = False
# Создаем список для хранения истории сообщений
# messages = [HumanMessage(content="Начнем процесс сбора информации о пользователе.")]
messages = [{'role': 'user', 'content': "Начнем процесс сбора информации о пользователе."}]

# Запускаем цикл взаимодействия с агентом
while not profile_saved:
    print(f'debug: before {profile_saved=}')
    # Запускаем агента с текущей историей сообщений
    response = agent.invoke({"messages": messages})
    
    
    # Добавляем ответ агента в историю
    messages.append(AIMessage(content=response["messages"][-1].content))
    
    # Выводим последний ответ агента
    print("\nАгент:", response["messages"][-1].content)
    print(f'after: {profile_saved=}')
    # Если профиль уже сохранен, выходим из цикла
    if profile_saved:
        break
        
    # Получаем ввод пользователя
    user_input = input("\nВаш ответ: ")
    if user_input == '_exit':
        print("exitting...")
        break
    
    print("\nВы:", user_input)
    # Добавляем сообщение пользователя в историю
    messages.append(HumanMessage(content=user_input))

print("\nПроцесс сбора данных завершен!")

debug: before profile_saved=False

Агент: Пожалуйста, представьтесь. What is your name?
after: profile_saved=False

Ваш ответ: 
Вы: Меня зовут Джонни Кейдж
debug: before profile_saved=False

Агент: Отлично, Джонни Кейдж! А теперь расскажи, сколько тебе лет?
after: profile_saved=False

Ваш ответ: 
Вы: 50653 года и 45 месяцев
debug: before profile_saved=False

Агент: Кажется, ты шутишь? Давай-ка введем реалистичный возраст — целое число лет, например "30". Сколько тебе на самом деле лет?
after: profile_saved=False

Ваш ответ: 
Вы: Понятно все с тобой...
debug: before profile_saved=False

Агент: Джонни, давай попробуем ещё раз! Мне真的 нужен твой реальный возраст — это просто число, например **28**. Помни, должно быть целое число больше 0. Сколько тебе лет?
after: profile_saved=False

Ваш ответ: 
Вы: Ладно. По вашим глупым земным меркам мне 26
debug: before profile_saved=False

Агент: Отлично, Джонни! Теперь поделись, **в каком городе** ты живешь и который тебе особенно нравится? Например: 